In [ ]:
using JuMP
# using Ipopt
using MadNLP
using ProgressMeter
using CairoMakie

In [ ]:
num_triangs_hor = 4 # choose even
num_triangs_ver = 4 # choose even
num_triangs = 2*num_triangs_hor * num_triangs_ver
num_edges = num_triangs_hor * (num_triangs_ver + 1) + num_triangs_ver * (2*num_triangs_hor + 1)
num_free_verts_hor = num_triangs_hor - 1 # ignore leftmost and rightmost vertices
num_free_verts_ver = num_triangs_ver + 1
num_verts_hor = num_free_verts_hor + 2
num_verts_ver = num_free_verts_ver
fps = 120
animation_fps = 60
animation_scale = 4
triang_side_length = 1
triang_height = sqrt(3)/2 * triang_side_length
pref_dist = triang_side_length
break_dist = 2*pref_dist
start_width = num_triangs_hor*triang_side_length
max_pull = 1.4*break_dist - pref_dist
max_push = 0.6*pref_dist
min_x = 0
max_x = start_width + max_pull + triang_side_length/2
min_y = -triang_height
max_y = (num_triangs_ver + 1)*triang_height
pot_min = -1.
diss_coeff = 0.5
time_pull = 2.
l2_dissipation = true # if false => Kelvin-Voigt
soft_max_alpha = 5
time_horizon = 5*time_pull
animation_width = 800
animation_aspect = (max_x - min_x) / (max_y - min_y)
animation_height = animation_width / animation_aspect
dotsize = 6
linewidth = 1
hidpi_scaling = 2
file_name = "square"
fontsize = 16
only_video = true; # if false generate a folder of snapshots for each frame

In [ ]:
function lennard_jones(dist_sq)
    # Lennard-jones of squared distances.
    # The factor cbrt(2) assures that the global minimum 
    # is attained at cur_dist == pref_dist
    q = pref_dist^2/(cbrt(2)*dist_sq)
    -4*pot_min*(q^6 - q^3)
    # alternative power
    # q = pref_dist^2/(2*dist_sq)
    # -4*pot_min*(q^2 - q)
end

In [ ]:
function dirichlet(t)
    t = t % (4*time_pull)
    if t <= 2*time_pull
        return start_width + max_pull*sin(pi/2*t/time_pull)
    else
        return start_width + max_push*sin(pi/2*t/time_pull)
    end
end

ts = range(0, time_horizon, 1000)
lines(ts, dirichlet.(ts))

In [ ]:
function get_edges(verts)
    edges = Matrix{Tuple{Any, Any}}(undef, num_edges, 2)
    edge_id = 1
    # add horizontal edges
    for i in 1:num_verts_hor-1
        for j in 1:num_verts_ver
            edges[edge_id, 1] = verts[i,j]
            edges[edge_id, 2] = verts[i+1,j]
            edge_id += 1
        end
    end
    # right-leaning odd rows
    for i in 1:num_verts_hor
        for j in 1:2:num_verts_ver-1
            edges[edge_id, 1] = verts[i,j]
            edges[edge_id, 2] = verts[i,j+1]
            edge_id += 1
        end
    end
    # right-leaning even rows
    for i in 1:num_verts_hor-1
        for j in 2:2:num_verts_ver-1
            edges[edge_id, 1] = verts[i,j]
            edges[edge_id, 2] = verts[i+1,j+1]
            edge_id += 1
        end
    end
    # left-leaning odd rows
    for i in 2:num_verts_hor
        for j in 1:2:num_verts_ver-1
            edges[edge_id, 1] = verts[i,j]
            edges[edge_id, 2] = verts[i-1,j+1]
            edge_id += 1
        end
    end
    # left-leaning even rows
    for i in 1:num_verts_hor
        for j in 2:2:num_verts_ver-1
            edges[edge_id, 1] = verts[i,j]
            edges[edge_id, 2] = verts[i,j+1]
            edge_id += 1
        end
    end 
    edges
end

In [ ]:
function get_triangles(verts)
    triangs = Matrix{Tuple{Any, Any}}(undef, num_triangs, 3)
    triang_id = 1
    # upward-pointing odd rows
    for i in 1:num_verts_hor-1
        for j in 1:2:num_verts_ver-1
            triangs[triang_id, 1] = verts[i,j]
            triangs[triang_id, 2] = verts[i+1,j]
            triangs[triang_id, 3] = verts[i,j+1]
            triang_id += 1
        end
    end
    # upward-pointing even rows
    for i in 1:num_verts_hor-1
        for j in 2:2:num_verts_ver-1
            triangs[triang_id, 1] = verts[i,j]
            triangs[triang_id, 2] = verts[i+1,j]
            triangs[triang_id, 3] = verts[i+1,j+1]
            triang_id += 1
        end
    end
    # downward-pointing odd rows
    for i in 1:num_verts_hor-1
        for j in 3:2:num_verts_ver
            triangs[triang_id, 1] = verts[i,j]
            triangs[triang_id, 2] = verts[i,j-1]
            triangs[triang_id, 3] = verts[i+1,j]
            triang_id += 1
        end
    end
    # downward-pointing odd rows
    for i in 1:num_verts_hor-1
        for j in 2:2:num_verts_ver
            triangs[triang_id, 1] = verts[i,j]
            triangs[triang_id, 2] = verts[i+1,j-1]
            triangs[triang_id, 3] = verts[i+1,j]
            triang_id += 1
        end
    end
    triangs
end

In [ ]:
function vector_to_values(vec)
    broadcast((p) -> value.(p), vec)
end

In [ ]:
function plot_step!(ax, vertices, edges, triangles)
    plot_triangles!(ax, triangles)
    plot_edges!(ax, edges)
    plot_vertices!(ax, vertices)
end

function plot_step(step)
    fig = Figure(
        resolution=(
            animation_scale*animation_width,
            animation_scale*animation_height
        ), fontsize=animation_scale*fontsize
    )
    ax = Axis(
        fig[1, 1],
        limits=(min_x-0.25, max_x+0.25, min_y, max_y),
        aspect=animation_aspect,
    )
    
    plot_step!(ax, vector_to_values.(step)...)

    fig
end

function plot_vertices!(ax, vertices)
    vertices = vertices[:]
    x = [v[1] for v in vertices]
    y = [v[2] for v in vertices]
    scatter!(ax, x, y, color=:blue, markersize=animation_scale*dotsize)
end

function plot_edges!(ax, edges)
    num_edges = size(edges)[1]
    for i in 1:num_edges
        v1, v2 = edges[i, :]
        x1, y1 = v1
        x2, y2 = v2
        lines!(
            ax, [x1, x2], [y1, y2],
            color=:green,
            linewidth=animation_scale*linewidth,
#             linestyle=:dash
        )
    end
end

function plot_triangles!(ax, triangles)
    num_triags = size(triangles)[1]

    for i in 1:num_triags
        poly!(ax, Tuple.(triangles[i, :]), color=(:pink, 0.5))
    end
end

In [ ]:
minmove = Model(
    ()->MadNLP.Optimizer(
        print_level=MadNLP.WARN,
        blas_num_threads=8,
        # acceptable_tol=1e-8,
        # max_iter=1000
    )
)

# minmove = Model(Ipopt.Optimizer)
# set_optimizer_attribute(minmove, "max_cpu_time", 60.0)
# set_optimizer_attribute(minmove, "print_level", 0)

step = 2
prev_L = num_triangs_hor*triang_side_length
L = dirichlet((step-1)/fps)

@NLparameter(
    minmove,
    prev_x[i=1:num_free_verts_hor,j=1:num_free_verts_ver] ==
        i*triang_side_length + ((j+1) % 2)*triang_side_length/2
)
@NLparameter(
    minmove,
    prev_y[i=1:num_free_verts_hor,j=1:num_free_verts_ver] ==
        (j-1)*triang_height
)

delta_L = L - prev_L
# apply uniform correction
@variable(
    minmove,
    min_x <= x[i=1:num_free_verts_hor,j=1:num_free_verts_ver] <= max_x,
    start = value(prev_x[i,j]) + delta_L/num_free_verts_hor
)
@variable(
    minmove,
    min_y <= y[i=1:num_free_verts_hor,j=1:num_free_verts_ver] <= max_y,
    start = value(prev_y[i,j])
)

# Dirichlet condition
@NLparameter(
    minmove,
    prev_bdry_x[j=1:num_free_verts_ver] == prev_L + ((j+1) % 2)*triang_side_length/2
)
@NLparameter(
    minmove,
    bdry_x[j=1:num_free_verts_ver] == L + ((j+1) % 2)*triang_side_length/2
)

# vertices 
prev_vertices = Matrix{Any}(undef, num_verts_hor, num_verts_ver)
vertices = Matrix{Any}(undef, num_verts_hor, num_verts_ver)
# leftmost (fixed)
for j in 1:num_verts_ver
    prev_vertices[1,j] = (
        ((j+1) % 2)*triang_side_length/2,
        (j-1)*triang_height
    )
    vertices[1,j] = prev_vertices[1,j]
end
# middle (free)
for i in 1:num_free_verts_hor
    for j in 1:num_free_verts_ver
        prev_vertices[i+1,j] = (
            prev_x[i,j],
            prev_y[i,j]
        )
        vertices[i+1,j] = (
            x[i,j],
            y[i,j]
        )
    end
end
# rightmost (driven by the boundary condition)
for j in 1:num_verts_ver
    prev_vertices[num_verts_hor, j] = (
        prev_bdry_x[j],
        (j-1)*triang_height
    )
    vertices[num_verts_hor, j] = (
        bdry_x[j],
        (j-1)*triang_height
    )
end

prev_edges = get_edges(prev_vertices)
edges = get_edges(vertices)

prev_triangles = get_triangles(prev_vertices)
triangles = get_triangles(vertices)

@NLexpression(
    minmove,
    dissipation,
    sum(
        (x[i,j]-prev_x[i,j])^2 + (y[i,j]-prev_y[i,j])^2
        for i in 1:num_free_verts_hor, j in 1:num_free_verts_ver
    )
)

@expression(minmove, v1[i=1:num_edges], edges[i,1])
@expression(minmove, v2[i=1:num_edges], edges[i,2])
@NLexpression(
    minmove,
    dist_sq[i=1:num_edges],
    (v2[i][1] - v1[i][1])^2 + (v2[i][2] - v1[i][2])^2
)
# we implicitely assume that the simulation starts
# in a globally optimal state
@NLparameter(minmove, max_dist_sq[i=1:num_edges] == pref_dist)
@NLparameter(minmove, broken[i=1:num_edges] == 0.)

register(minmove, :W, 1, lennard_jones, autodiff = true)
# no memory for the moment
@NLexpression(
    minmove,
    energy,
    sum((1-broken[i])*W(dist_sq[i]) + broken[i]*(
        (
            W(dist_sq[i])*exp(soft_max_alpha*W(dist_sq[i])) +
            W(max_dist_sq[i])*exp(soft_max_alpha*W(max_dist_sq[i]))
        ) / (exp(soft_max_alpha*W(dist_sq[i])) + exp(soft_max_alpha*W(max_dist_sq[i])))
    ) for i in 1:num_edges)
)

@NLobjective(
    minmove,
    Min,
    energy + diss_coeff*fps*dissipation
)

optimize!(minmove)

steps = [vector_to_values.((prev_vertices, prev_edges, prev_triangles))]
push!(steps, vector_to_values.((vertices, edges, triangles)));

In [ ]:
plot_step(steps[2])

In [ ]:
@showprogress "Computing minmoves..." for step in 3:fps*time_horizon
    # update memory variables
    last_dist_sq = value.(dist_sq)
    new_max_dist_sq = max.(value.(max_dist_sq), last_dist_sq)
    set_value.(max_dist_sq, new_max_dist_sq)
    
    # check for broken links
    new_broken = new_max_dist_sq .> break_dist^2
    set_value.(broken, new_broken)
    
    # new -> old
    set_value.(prev_y, value.(y))
    set_value.(prev_x, value.(x))
    prev_L = L
    L = dirichlet((step-1)/fps)
    delta_L = L - prev_L
    
    # update right boundary
    set_value.(prev_bdry_x, value.(bdry_x))
    set_value.(bdry_x, value.(bdry_x) .+ delta_L)

    # update start values
    new_start_x = value.(x) .+ delta_L/num_free_verts_hor
    new_start_y = value.(y)
    set_start_value.(x, new_start_x)
    set_start_value.(y, new_start_y)

    optimize!(minmove)

    push!(steps, vector_to_values.((vertices, edges, triangles)))
end

In [ ]:
length(steps)

In [ ]:
sum(value.(broken))

In [ ]:
plot_step(steps[400])

In [ ]:
function animate_steps(steps)
    experiments_folder = "experiments"
    if !ispath(experiments_folder)
        mkdir(experiments_folder)
    end
    if !only_video
        snapshots_folder = "$experiments_folder/$file_name"
        if !ispath(snapshots_folder)
            mkdir(snapshots_folder)
        end
    end
    
    fig = Figure(
        resolution=(
            animation_scale*animation_width,
            animation_scale*animation_height
        ), fontsize=animation_scale*fontsize
    )
    ax = Axis(
        fig[1, 1],
        limits=(min_x-0.25, max_x+0.25, min_y, max_y),
        aspect=animation_aspect,
    )
    
    frame = 1
    record(fig, "experiments/$file_name.mp4", steps; framerate=animation_fps) do (step)
        empty!(ax)
        plot_step!(ax, step...)
        if !only_video
            save("$snapshots_folder/$(file_name)_$frame.png", fig)
        end
        frame += 1
    end
end

In [ ]:
# animate_steps(steps[1:4:end])